In [2]:
import math as m
import sys
import numpy as np
from numpy import cos
from numpy import pi
np.set_printoptions(threshold=sys.maxsize)
from scipy import linalg as LA

In [3]:
basis_element = {1 : (0,0,1,1), 2 : (0,1,0,1), 3 : (0,1,1,0), 4 : (1,0,0,1), 5 : (1,0,1,0), 6 : (1,1,0,0)}
basis = {}
for i in range(1, len(basis_element.keys())+1):
    for j in range(1, len(basis_element.keys())+1):
        basis.update({(len(basis_element)*(i-1)+j) : basis_element[i]+basis_element[j]})
'''        
basis_element = {1: (0,0,0,1), 2: (0,0,1,0), 3: (0,1,0,0), 4:(1,0,0,0)}
basis = {}
for i in range(1, len(basis_element.keys())+1):
    for j in range(1, len(basis_element.keys())+1):
        basis.update({(len(basis_element)*(i-1)+j) : basis_element[i]+basis_element[j]})
'''

'        \nbasis_element = {1: (0,0,0,1), 2: (0,0,1,0), 3: (0,1,0,0), 4:(1,0,0,0)}\nbasis = {}\nfor i in range(1, len(basis_element.keys())+1):\n    for j in range(1, len(basis_element.keys())+1):\n        basis.update({(len(basis_element)*(i-1)+j) : basis_element[i]+basis_element[j]})\n'

In [4]:
def des(vector, x): #annihiltion operator
    old_sign = 1
    new_sign = 1
    vector_temp = list(vector)
    for i in range(8-x):
        if vector_temp[i] == 1:
            #print('found')
            #print(i)
            new_sign = old_sign * (-1)
            old_sign = new_sign         
    vector_temp[-x] = 0
    #print(count)
    #if count%2 == 1:
        #sign = -1
    #print(sign)
    return [new_sign, vector_temp]
    
def crt(vector, x): #creation operator
    old_sign = 1
    new_sign = 1
    vector_temp = list(vector)
    for i in range(8-x):
        if vector_temp[i] == 1:
            new_sign = old_sign * (-1)
            old_sign = new_sign
    vector_temp[-x] = 1
    return [new_sign, vector_temp]

In [5]:
def tterm(vector): #construct the kinetic energy term
    epsilon = 0.0
    vector_temp = list(vector)
    for i in range(len(vector_temp)):
        if vector_temp[i] == 1:
            k = (3 - i) * pi / 2
            epsilon += -2.0 * cos(k)
    return epsilon

In [6]:
def occup(vector): #stores the indices of occupied up's
    occuplist = []
    vector_temp = vector
    for i in range(1,5):
        if vector_temp[-i] == 1:
            occuplist.append(i)
    return occuplist

def unoccup(vector): #stores the indices of unoccupied up's
    unoccuplist = []
    vector_temp = vector
    for i in range(1,5):
        if vector_temp[-i] == 0:
            unoccuplist.append(i)
    return unoccuplist

def occdn(vector): #stores the indices of occupied down's
    occdnlist = []
    vector_temp = vector
    for i in range(5,9):
        if vector_temp[-i] == 1:
            occdnlist.append(i)
    return occdnlist

def unoccdn(vector): #stores the indices of unoccupied down's
    unoccdnlist = []
    vector_temp = vector
    for i in range(5,9):
        if vector_temp[-i] == 0:
            unoccdnlist.append(i)
    return unoccdnlist

In [7]:
def uterm(vector,j,k,l): #construct the interaction term
    i = k+l-j+4
    vector_temp = list(vector)
    temp4 = [0, vector_temp]
    if (i !=8 and i > 4):
        i = i % 4
    if i == 8:
        i = i - 4
    if (i != k):
        #print('aha')
        vector_temp = list(vector)
        temp1 = des(vector_temp,l)
        #print(temp1)
        temp2 = [des(temp1[1],k)[0] * temp1[0], des(temp1[1],k)[1]]
        #print(temp2)
        temp3 = [crt(temp2[1],j)[0] * temp2[0], crt(temp2[1],j)[1]]
        #print(temp3)
        temp4 = [crt(temp3[1],i)[0] * temp3[0], crt(temp3[1],i)[1]]
        #print(temp4)
        for index, val in basis.items():
            if val == tuple(temp4[1]):
                temp4 = [temp4[0], str(index)]
    return temp4

def row(x): #spit out the rows
    vrow = []
    for l in occdn(basis[x]):
        for k in occup(basis[x]):
            for j in unoccdn(basis[x]):
                #print(uterm(basis[x],j,k,l)[1])
                if type(uterm(basis[x],j,k,l)[1]) == str:
                    #print(j,k,l)
                    vrow.append(uterm(basis[x],j,k,l))
                
    return vrow
def uent(x,y): #project the rows into cols to get each entry
    value = 0
    for i in range(len(row(x))):
        if y == int(row(x)[i][1]):
            value = row(x)[i][0]
    return value

In [8]:
#actual code starts here
nelec = 4
mu = 0
def hamm(U):
    umatrix = np.zeros([len(basis),len(basis)],dtype = float)
    tmatrix = np.zeros([len(basis),len(basis)],dtype = float)
    for i in range(len(basis)):
        tmatrix[i,i] = tterm(basis[i+1]) - mu * nelec / 2.0 + U
        #2 is the number Jim chose for the electron-hole model
    for i in range(len(basis)):
        for j in range(len(basis)):
            umatrix[i,j] = uent(i+1,j+1) * U / nelec 
    hammatrix = tmatrix - umatrix
    return hammatrix

In [10]:
'''
interaction = 0.2
eigenvalues, eigenvectors = LA.eigh(hamm(interaction))
eigenvalues = eigenvalues.real
eigenvectors = eigenvectors.real

idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[idx]
print(eigenvalues)
print(eigenvectors)
'''

'\ninteraction = 0.2\neigenvalues, eigenvectors = LA.eigh(hamm(interaction))\neigenvalues = eigenvalues.real\neigenvectors = eigenvectors.real\n\nidx = eigenvalues.argsort()[::-1]\neigenvalues = eigenvalues[idx]\neigenvectors = eigenvectors[idx]\nprint(eigenvalues)\nprint(eigenvectors)\n'